# Quick EDA - Christmas Tree Packing

In [1]:
import pandas as pd
import numpy as np

# Load sample submission
df = pd.read_csv('/home/data/sample_submission.csv')
print(f'Shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
df.head(10)

Shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']


,id,x,y,deg
0,001_0,s0.0,s0.0,s90.0
1,002_0,s0.0,s0.0,s90.0
2,002_1,s0.202736,s-0.511271,s90.0
3,003_0,s0.0,s0.0,s90.0
4,003_1,s0.202736,s-0.511271,s90.0
5,003_2,s0.5206,s0.177413,s180.0
6,004_0,s0.0,s0.0,s90.0
7,004_1,s0.202736,s-0.511271,s90.0
8,004_2,s0.5206,s0.177413,s180.0
9,004_3,s-0.818657,s-0.228694,s180.0


In [2]:
# Calculate total number of trees: sum(1 to 200) = 200*201/2 = 20100
print(f"Expected rows: {200*201//2}")
print(f"Actual rows: {len(df)}")

# Parse the data
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))
df['x_val'] = df['x'].str[1:].astype(float)
df['y_val'] = df['y'].str[1:].astype(float)
df['deg_val'] = df['deg'].str[1:].astype(float)

print(f"\nN values range: {df['n'].min()} to {df['n'].max()}")
print(f"\nSample parsed data:")
df[['id', 'n', 'tree_idx', 'x_val', 'y_val', 'deg_val']].head(10)

Expected rows: 20100
Actual rows: 20100

N values range: 1 to 200

Sample parsed data:


,id,n,tree_idx,x_val,y_val,deg_val
0,001_0,1,0,0.000000,0.000000,90.0
1,002_0,2,0,0.000000,0.000000,90.0
2,002_1,2,1,0.202736,-0.511271,90.0
3,003_0,3,0,0.000000,0.000000,90.0
4,003_1,3,1,0.202736,-0.511271,90.0
5,003_2,3,2,0.520600,0.177413,180.0
6,004_0,4,0,0.000000,0.000000,90.0
7,004_1,4,1,0.202736,-0.511271,90.0
8,004_2,4,2,0.520600,0.177413,180.0
9,004_3,4,3,-0.818657,-0.228694,180.0
